In [2]:
# pip install ultralytics>=8.2.95
# ! pip install scikit-learn
# ! pip uninstall ultralytics
# ! pip install ultralytics


In [2]:
# Import library
import os
import xml.etree.ElementTree as ET
# from PIL import Image
import pandas as pd
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.metrics import confusion_matrix
from PIL import Image
import shutil
# from sklearn.model_selection import train_test_split

In [3]:
# Define class names and their corresponding IDs
class_names = {'with_mask': 0, 'without_mask': 1, 'mask_weared_incorrect': 2}

In [3]:
annotations_dir = 'datasets/annotations'
images_dir = 'datasets/images'
output_dir = 'datasets/dataset'
os.makedirs(f"{output_dir}/images", exist_ok=True)
os.makedirs(f"{output_dir}/labels", exist_ok=True)

In [4]:
# Define paths
images_dir = 'datasets/images'
annotations_dir = 'datasets/annotations'

train_images_dir = 'datasets/train/images'
train_annotations_dir = 'datasets/train/annotations'
val_images_dir = 'datasets/val/images'
val_annotations_dir = 'datasets/val/annotations'
test_images_dir = 'datasets/test/images'
test_annotations_dir = 'datasets/test/annotations'

# Create directories if they don't exist
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_annotations_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_annotations_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_annotations_dir, exist_ok=True)

# Get list of all image files
image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]

# Split the dataset into train and temp (which will be further split into val and test)
train_files, temp_files = train_test_split(image_files, test_size=0.3, random_state=42)

# Split the temp dataset into val and test
val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

# Copy files to train directories
for file in train_files:
    shutil.copy(os.path.join(images_dir, file), os.path.join(train_images_dir, file))
    shutil.copy(os.path.join(annotations_dir, file.replace('.png', '.xml')), os.path.join(train_annotations_dir, file.replace('.png', '.xml')))

# Copy files to val directories
for file in val_files:
    shutil.copy(os.path.join(images_dir, file), os.path.join(val_images_dir, file))
    shutil.copy(os.path.join(annotations_dir, file.replace('.png', '.xml')), os.path.join(val_annotations_dir, file.replace('.png', '.xml')))

# Copy files to test directories
for file in test_files:
    shutil.copy(os.path.join(images_dir, file), os.path.join(test_images_dir, file))
    shutil.copy(os.path.join(annotations_dir, file.replace('.png', '.xml')), os.path.join(test_annotations_dir, file.replace('.png', '.xml')))

print("Dataset split into training, validation, and test sets.")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'datasets/images'

In [ ]:
# Create yaml file for Yolo model training
yaml_text = """
train: datasets/train/images
test: datasets/test/images
val: datasets/train/images
nc: 3  # number of classes (with_mask, without_mask, mask_weared_incorrect)
names: ['with_mask', 'without_mask', 'mask_weared_incorrect']
"""

with open("data.yaml", 'w') as file:
    file.write(yaml_text)

# Read the yaml file to see changes are reflecting correctly
with open("data.yaml") as f:
    contents = f.read()
    print(contents)

In [ ]:
# convert annotation to labels for yolo
def convert_xml_to_yolo(xml_file, output_dir, dtatype):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    image_id = os.path.splitext(os.path.basename(xml_file))[0]
    yolo_annotation = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name not in class_names:
            print(f"Warning: Class '{class_name}' not found in class_names. Skipping.")
            continue
        class_id = class_names[class_name]
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        
        x_center = (xmin + xmax) / 2.0
        y_center = (ymin + ymax) / 2.0
        width = xmax - xmin
        height = ymax - ymin
        
        # Load the image from folder
        img_path = os.path.join('datasets/' + dtatype + '/images', f"{image_id}.png")
        with Image.open(img_path) as img:
            img_width, img_height = img.size
        
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height
        
        yolo_annotation.append(f"{class_id} {x_center} {y_center} {width} {height}")

    with open(os.path.join(output_dir, f"{image_id}.txt"), 'w') as f:
        f.write("\n".join(yolo_annotation))

# Convert all XML files in the annotations directory
data_list = ['train', 'test', 'val']
for i in data_list:
    annotations_dir = 'datasets/' + i + '/annotations'
    output_dir = 'datasets/' + i + '/labels'
    os.makedirs(output_dir, exist_ok=True)

    # conversion
    for xml_file in os.listdir(annotations_dir):
        if xml_file.endswith('.xml'):
            convert_xml_to_yolo(os.path.join(annotations_dir, xml_file), output_dir, i)

print("Conversion completed successfully.")

In [ ]:
# Load a YOLOv8 model
model = YOLO('yolov8n.pt') 

In [ ]:
# Fine-tune the model
n_epoch = 10
l_r = 0.001
model.train(data='datasets/data.yaml', 
            epochs=n_epoch, 
            imgsz=640, 
            lr0=l_r)

# Save the trained model
model.save('trained_weights/exp1_yolov8n_trained.pt')

Training completed

Latency Training

In [ ]:
# Load the YOLOv8 model
model = YOLO("yolov8s.pt")  # Pre-trained YOLOv8 small model

# Train the model
model.train(
    data="datasets/data.yaml",
    epochs=100,
    imgsz=640,
    batch=2,
    name="yolov8_mask_detection",
    project="mask-detection-project"
)


Ultralytics 8.3.52  Python-3.10.16 torch-2.5.1+cpu CPU (Intel Core(TM) i5-7200U 2.50GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=datasets/data.yaml, epochs=100, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=mask-detection-project, name=yolov8_mask_detection4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

train: Scanning E:\Face-mask-detection\datasets\train\labels... 0 images, 682 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:15<00:00, 44.89it/s]

train: WARNING  No labels found in E:\Face-mask-detection\datasets\train\labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
train: New cache created: E:\Face-mask-detection\datasets\train\labels.cache
WARNING  No labels found in E:\Face-mask-detection\datasets\train\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning E:\Face-mask-detection\datasets\val\labels... 0 images, 85 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:01<00:00, 50.72it/s]

val: WARNING  No labels found in E:\Face-mask-detection\datasets\val\labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: New cache created: E:\Face-mask-detection\datasets\val\labels.cache
WARNING  No labels found in E:\Face-mask-detection\datasets\val\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to mask-detection-project\yolov8_mask_detection4\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to mask-detection-project\yolov8_mask_detection4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G          0      5.393          0          0        640: 100%|██████████| 341/341 [18:43<00:00,  3.29s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:39<00:00,  1.80s/it]

                   all         85          0          0          0          0          0
WARNING  no labels found in detect set, can not compute metrics without labels



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/100         0G          0     0.2289          0          0        640:  46%|████▌     | 156/341 [08:16<12:21,  4.01s/it]

In [9]:
pip install --upgrade ultralytics


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: ultralytics in c:\users\kasish\anaconda3\envs\yolov7\lib\site-packages (8.3.52)

